# Заполним пропуски

Пропуски есть в near_metro и class

# Near metro
## Заполним через api yandex "поиск по организациям"

In [4]:
import json
import geopy.distance as distance
import pandas as pd
import requests
data = pd.read_csv('../result.csv')

In [8]:
def get_near_metro(lon, lat, api_key):
    req_text = 'https://search-maps.yandex.ru/v1/?text=метро&ll=' + str(lon) + "," + str(lat) + '&lang=ru_RU&results=1&apikey=' + api_key
    ans = json.loads(requests.get(req_text).text)
    try:
        metro_lon, metro_lat = ans['features'][0]['geometry']['coordinates']
        result = distance.vincenty((lon, lat), (metro_lon, metro_lat)).m
    except:
        result = np.nan
    return result

In [6]:
null_metro = data[data['near_metro'].isnull()]

In [9]:
null_metro.head(2)

,Unnamed: 0,size,class,address,price,pledge,descr,metros,near_metro,url,parking,internet,meal,security,comfort,lat,lon,distance
39,1002926616,57.0,NaN,"Петровско-Разумовская, Лихоборская наб 7",685,0,Сдаются офис склад на территории бывшего про...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,0,0,55.854411,37.529070,0.20401
56,1690633446,39.0,NaN,"Автозаводская, Кожуховский 2-й проезд 29к2с...",1085,0,ДО КОНЦА 2018 ГОДА МЕСЯЦ АРЕНДЫ ЭТОГО ПОМЕЩЕ...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,1,1,55.698391,37.658248,0.09784


In [11]:
with open('../apies/yandex_organization_search_api.txt') as f:
    apikey = f.readline()

Через API можно отправлять только 500 запросов, раз в сутки

In [12]:
# for index, row in null_metro.iterrows():
#     null_metro.at[index, 'near_metro'] = get_near_metro(row['lon'], row['lat'], apikey)

In [13]:
# data[data['near_metro'].isnull()] = null_metro
# data.to_csv('got_near_metro.csv')

In [17]:
data = pd.read_csv('got_near_metro.csv')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3155 entries, 0 to 3154
Data columns (total 14 columns):
Unnamed: 0    3155 non-null int64
size          3155 non-null float64
class         1954 non-null object
price         3155 non-null int64
pledge        3155 non-null int64
near_metro    3155 non-null float64
parking       3155 non-null int64
internet      3155 non-null int64
meal          3155 non-null int64
security      3155 non-null int64
comfort       3155 non-null int64
lat           3155 non-null float64
lon           3155 non-null float64
distance      3155 non-null float64
dtypes: float64(5), int64(8), object(1)
memory usage: 345.2+ KB


# Class

Нужно разделить на обучающую и целевую выборку.

In [35]:
null_classes = data[data['class'].isnull()]
not_null_classes = data[~data['class'].isnull()]

Заполним Nan-ы knn - ом.

In [36]:
null_classes.head(5)

,Unnamed: 0,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,0,28.8,NaN,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,1,588.2,NaN,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,2,45.0,NaN,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,3,100.0,NaN,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,4,190.0,NaN,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [60]:
scaler = StandardScaler()

In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
X_data = not_null_classes.drop(['Unnamed: 0', 'class'], axis=1)
y_data = not_null_classes['class'].copy()
scaler.fit(X_data)
X_data = scaler.transform(X_data)
X_data

array([[-0.29401913, -0.06195927, -0.23075499, ..., -1.70871113,
        -1.94841224,  2.01179132],
       [-0.07759374,  0.86149217,  0.07749525, ...,  0.42149252,
         0.67963153, -0.37195182],
       [-0.31546844,  0.01590425, -0.20349912, ..., -0.38458446,
        -1.59500452,  1.26580109],
       ..., 
       [-0.30155537, -0.02358545, -0.25606401, ..., -0.60492427,
         0.42777408, -0.71009231],
       [ 0.07023254, -0.06493495, -0.29500097, ..., -2.01991615,
        -0.29031496, -0.10876948],
       [ 1.53110396, -0.05978951, -0.24438293, ...,  0.58340385,
         0.63852968, -0.412746  ]])

In [66]:
from sklearn.model_selection import cross_val_score
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X_data, y_data, cv=15)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.71428571  0.68421053  0.6969697   0.75757576  0.70229008  0.63358779
  0.73076923  0.75193798  0.66666667  0.65891473  0.69767442  0.72868217
  0.72868217  0.72868217  0.82170543]
Accuracy: 0.71 (+/- 0.09)


In [67]:
clf.fit(X_data, y_data)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [70]:
Target_data = null_classes.drop(['Unnamed: 0', 'class'], axis=1)
Target_data = pd.DataFrame(scaler.transform(Target_data))

In [71]:
Target_data['class'] = clf.predict(Target_data)

In [72]:
Result = pd.concat([not_null_classes, Target_data])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [73]:
Result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3155 entries, 5 to 1200
Data columns (total 26 columns):
Unnamed: 0    1954 non-null float64
class         3155 non-null object
comfort       1954 non-null float64
distance      1954 non-null float64
internet      1954 non-null float64
lat           1954 non-null float64
lon           1954 non-null float64
meal          1954 non-null float64
near_metro    1954 non-null float64
parking       1954 non-null float64
pledge        1954 non-null float64
price         1954 non-null float64
security      1954 non-null float64
size          1954 non-null float64
0             1201 non-null float64
1             1201 non-null float64
2             1201 non-null float64
3             1201 non-null float64
4             1201 non-null float64
5             1201 non-null float64
6             1201 non-null float64
7             1201 non-null float64
8             1201 non-null float64
9             1201 non-null float64
10            1201 non-null fl